In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# Define data path
DATA_PATH = "data"
GRAM_DATA_PATH = "../Project/code/processed_data/gram"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targs = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targs) == len(seqs)

In [3]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        
        """
        TODO: Store `seqs`. to `self.x` and `hfs` to `self.y`.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        Do NOT permute the data.
        """
#         x = []
#         for i,patient in enumerate(seqs):
#             for j,visit in enumerate(patient):
#                 if j == len(patient) - 1:
#                     break
#                 x.append(visit)
#         y = []
#         for i,patient in enumerate(targets):
#             for j,visit in enumerate(patient):
#                 if j == len(patient) - 1:
#                     break
#                 y.append(patient[j+1])

        self.x = seqs
        self.y = targets
    
    def __len__(self):
        
        """
        TODO: Return the number of samples (i.e. patients).
        """
        
        return(len(self.x))
    
    def __getitem__(self, index):
        
        """
        TODO: Generates one sample of data.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        """
        return (self.x[index], self.y[index])

In [4]:
dataset = CustomDataset(seqs, targs)

In [5]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (# patiens) of type torch.float
    """
    sequences, targets = zip(*data)

#     y = torch.tensor(targets, dtype=torch.float)
#     import pdb; pdb.set_trace()
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    batch_num_categories = [len(visit) for patient in targets for visit in patient]
    global sub_categories
# #     import pdb; pdb.set_trace()
    num_categories = len(sub_categories)

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    max_num_categories = max(batch_num_categories)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    x_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    y = torch.zeros((num_patients, max_num_categories), dtype=torch.long)
    y_masks = torch.zeros((num_patients, max_num_categories), dtype=torch.bool)
#     import pdb; pdb.set_trace()
    for i_patient, patient in enumerate(sequences):   
        for j_visit, visit in enumerate(patient[:-1]):
#             x[i_patient, j_visit] = torch.Tensor(visit)
#             x_masks[i_patient, j_visit] = torch.Tensor(np.ones(num_codes, dtype=int))
#             if j_visit == len(patient) - 2:
#                 rev_visit = x_masks[i_patient].any(dim=1)
#                 rev_x[i_patient, rev_visit] = x[i_patient, rev_visit].flip(0)
#                 rev_x_masks[i_patient, rev_visit] = x_masks[i_patient, rev_visit].flip(0)
            for k_code, code in enumerate(visit):
                x[i_patient, j_visit, k_code] = code
                x_masks[i_patient, j_visit, k_code] = 1

    for i_patient, patient in enumerate(targets):
        last_visit = patient[-1]
        y[i_patient,:len(last_visit)] = torch.LongTensor(last_visit)
        y_masks[i_patient,:len(last_visit)] = torch.BoolTensor(np.ones(len(last_visit)))
#         for j_visit, visit in enumerate(patient[-1:]):
#             for k_code, code in enumerate(visit):
#                 y[i_patient, j_visit, k_code] = code
#                 y_masks[i_patient, j_visit, k_code] = 1
#             y[i_patient, j_visit] = torch.Tensor(visit)
#             y_masks[i_patient, j_visit] = torch.Tensor(np.ones(num_codes, dtype=int))
    
    
    return x, x_masks, y, y_masks

In [6]:
#         x = []
#         for i,patient in enumerate(seqs):
#             for j,visit in enumerate(patient):
#                 if j == len(patient) - 1:
#                     break
#                 x.append(visit)
#         y = []
#         for i,patient in enumerate(targets):
#             for j,visit in enumerate(patient):
#                 if j == len(patient) - 1:
#                     break
#                 y.append(patient[j+1])

In [7]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [8]:
from torch.utils.data.dataset import random_split

train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [9]:
from torch.utils.data import DataLoader

def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    
    '''
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, val_loader: train and validation dataloaders
    '''
    
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [10]:
def sum_embeddings_with_mask(x, masks):
    """
    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    x[~masks] = 0
    return x.sum(2)

In [11]:
def get_last_visit(hidden_states, masks):
    """
    Arguments:
        hidden_states: the hidden states of each visit of shape (batch_size, # visits, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        last_hidden_state: the hidden state for the last true visit of shape (batch_size, embedding_dim)
    """
    idx_vector = masks.any(dim=2).sum(1) - 1
    p_idx = torch.arange(0,len(hidden_states), dtype=torch.int64)
    last_hidden_state = hidden_states[p_idx,idx_vector]
    return last_hidden_state

In [12]:
class BaselineRNN(nn.Module):
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
        """
        
        self.embedding = nn.Embedding(num_codes, embedding_dim=128)
        self.rnn = nn.GRU(128, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, num_categories)
        self.softmax = nn.Softmax(dim=1)
        #self.sigmoid = nn.Sigmoid()
        
    
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            probs: probabilities of shape (batch_size)
        """
#         import pdb; pdb.set_trace()
        # 1. Pass the sequence through the embedding layer;
        x = self.embedding(x)
        # 2. Sum the embeddings for each diagnosis code up for a visit of a patient.
        x = sum_embeddings_with_mask(x, masks)
        
        # 3. Pass the embegginds through the RNN layer;
        output, _ = self.rnn(x)
        # 4. Obtain the hidden state at the last visit.
        true_h_n = get_last_visit(output, masks)
        
        # 6. Pass the hidden state through the linear and activation layers.
        #import pdb; pdb.set_trace()
        logits = self.fc(true_h_n)        
        probs = self.softmax(logits)
        #probs = self.sigmoid(logits)
        
#         probs = probs.reshape(probs.shape[0]*probs.shape[1], probs.shape[2])
#         y_masks = y_masks.reshape(y_masks.shape[0]*y_masks.shape[1], y_masks.shape[2])
#         probs = probs[y_masks.any(dim=1)]
        return logits
    

# load the model here
baseline_rnn = BaselineRNN(num_codes = len(codes), num_categories=len(sub_categories))
baseline_rnn

BaselineRNN(
  (embedding): Embedding(4903, 128)
  (rnn): GRU(128, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=184, bias=True)
  (softmax): Softmax(dim=1)
)

In [13]:
criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
#criterion = nn.BCEWithLogitsLoss()
#optimizer = torch.optim.Adam(baseline_rnn.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(baseline_rnn.parameters(), weight_decay=0.001)

In [14]:
def eval_model_old(model, test_loader, threshold=0.5, k=15, n=-1):
    
    """
   Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
    """
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    model.eval()
    with torch.no_grad():
        for x, masks, rev_x, rev_masks, y, y_masks in test_loader:
            #import pdb; pdb.set_trace()
            nn = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks, rev_x, rev_masks, y_masks)
#             num_categories = torch.count_nonzero(y, dim=2)
#             nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
            num_predictions = 0
            num_targets = 0
            all_predictions = []
            all_targets = []
            precision = 0
            total_precision = 0
            total_accuracy = 0
#             y_masks = y_masks.reshape(
#                 y_masks.shape[0] * y_masks.shape[1], y_masks.shape[2])
#             y = y.reshape(y.shape[0] * y.shape[1], y.shape[2])
#             y = y[y_masks.any(dim=1)]
#             y_masks = y_masks[y_masks.any(dim=1)]


#             v_idx = masks.any(dim=2)
#             v_idx = v_idx.sum(dim=1)
#             v_idx = v_idx.unsqueeze(-1)
#             v_idx = v_idx.repeat(1,y_hat.shape[2])
#             v_idx = v_idx.unsqueeze(1)
#             y_hat = torch.gather(y_hat,1,v_idx).squeeze()
            for i in range(k):
                
                visit_correct = 0
#                 y_true = nz_cols[nz_rows == i]

                y_true = y[i, y_masks[i]].unique()
                all_targets.extend(y_true.tolist())
                _, y_pred = torch.topk(y_hat[i], len(y_true))
                #y_pred = torch.nonzero(y_hat[0] > threshold).squeeze()
                if y_pred.numel() > 0:
                    try:
                        all_predictions.extend(y_pred.tolist())
                    except TypeError:
                        y_pred = [y_pred.tolist()]
                    all_predictions.extend(y_pred)
#                     for v in y_pred:
#                         if v in y_true:
#                             visit_correct += 1
                    for v in y_true:
                        if v in y_pred:
                            visit_correct += 1
                    num_predictions += len(y_pred)

                num_targets += len(y_true)
                precision += visit_correct / min(k, len(y_true))
                k_correct += visit_correct
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                total_precision += visit_precision
                total_accuracy += visit_accuracy
 #           import pdb; pdb.set_trace()
            precision_k = precision / k
#             precision_k1 = k_correct / min(k, num_targets)
            if num_predictions == 0:
                accuracy_k = 0
            else:
                accuracy_k = k_correct / num_predictions
            precision_k = total_precision / nn
            accuracy_k = total_accuracy / nn
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
#             y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
#             y_hat = (y_hat > 0.5).int()
#             y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
#             y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    """
    TODO:
        Calculate precision, recall, f1, and roc auc scores.
        Use `average='binary'` for calculating precision, recall, and fscore.
    """
#     p, r, f, s = precision_recall_fscore_support(y_true, y_pred, average='binary')
#     roc_auc = roc_auc_score(y_true, y_score)
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [26]:
def eval_model(model, test_loader, k=15, n=-1):
    
    """
    Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
        
    """
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y, y_masks in test_loader:
#             import pdb; pdb.set_trace()
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
#             num_labels = y_hat.shape[1]
#             num_categories = torch.count_nonzero(y, dim=1)
#             nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            y_multihot = indices_to_multihot(y, y_masks, y_hat)
            k_correct = 0
#             predictions = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
#                 y_true = nz_cols[nz_rows == i]
                y_true = y[i, y_masks[i]]
                _, y_pred = torch.topk(y_hat[i], k)
#                 for v in y_pred:
#                     if v in y_true:
#                         visit_correct += 1
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
#                 predictions += len(y_true)
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                #print(f'visit {i}: precision: {visit_precision:0.2f} accuracy: {visit_accuracy:0.2f}')
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            #import pdb; pdb.set_trace()
#             precision_k = precision / k
#             accuracy_k = k_correct / predictions
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
#             y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
#             y_hat = (y_hat > 0.5).int()
#             y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
#             y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
#     import pdb; pdb.set_trace()
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [27]:
def train(model, train_loader, test_loader, n_epochs):
    """
    Arguments:
        model: the RNN model
        train_loader: training dataloder
        val_loader: validation dataloader
        n_epochs: total number of epochs
    """
    #base_cpu, base_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, x_masks, y, y_masks in train_loader:
#             import pdb; pdb.set_trace()
            y_hat = model(x, x_masks)
#             import pdb; pdb.set_trace()
#             y[~y_masks] = criterion.ignore_index
#             last_y = y_masks.any(dim=2).sum(dim=1) - 1
#             indices = last_y.unsqueeze(-1)
#             indices = indices.repeat(1, y.shape[2])
#             indices = indices.unsqueeze(1)
#             y_filt = torch.gather(y, 1, indices)

#             n_visits = y_masks.any(dim=2).sum(dim=1)
#             for i_patient, j_visit in enumerate(n_visits):
#                 for visit in range(j_visit - 1):
#                     mask = y_masks[i_patient, visit+1]

#                     yh = y_hat[i_patient, visit]
#                     y_tmp = indices_to_multihot(
#                         y[i_patient, visit+1], mask, yh)
            
            # stack into visits
#             yh = y_hat.reshape(y_hat.shape[0] * y_hat.shape[1], y_hat.shape[2])
#             y_masks = y_masks.reshape(y_masks.shape[0] * y_masks.shape[1], y_masks.shape[2])
#             y = y.reshape(y.shape[0] * y.shape[1], y.shape[2])
        
                    
            
            y_mh = indices_to_multihot(y, y_masks, y_hat)
            loss = criterion(y_hat, y_mh)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print_cpu_usage()
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, test_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.2f}, accuracy@k{k}: {accuracy_k:.2f}')

In [28]:
def indices_to_multihot(indices, masks, y_hat):
#     import pdb; pdb.set_trace()
    #indices = indices[masks.any(dim=1)]
    multihot = torch.zeros_like(y_hat, dtype=torch.float)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx, y_hat.shape[1]).sum(0).float()
    return multihot

In [29]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [30]:
n_epochs = 100
%time train(baseline_rnn, train_loader, test_loader, n_epochs)

CPU: 14.57
RAM %: 59.7
Epoch: 1 	 Training Loss: 40.140969
Epoch: 1 	 Validation precision@k5: 0.70, accuracy@k5: 0.37
Epoch: 1 	 Validation precision@k10: 0.65, accuracy@k10: 0.55
Epoch: 1 	 Validation precision@k15: 0.69, accuracy@k15: 0.67
Epoch: 1 	 Validation precision@k20: 0.75, accuracy@k20: 0.75
Epoch: 1 	 Validation precision@k25: 0.80, accuracy@k25: 0.80
Epoch: 1 	 Validation precision@k30: 0.84, accuracy@k30: 0.84
CPU: 14.93
RAM %: 59.8
Epoch: 2 	 Training Loss: 39.573646
Epoch: 2 	 Validation precision@k5: 0.72, accuracy@k5: 0.38
Epoch: 2 	 Validation precision@k10: 0.67, accuracy@k10: 0.57
Epoch: 2 	 Validation precision@k15: 0.71, accuracy@k15: 0.68
Epoch: 2 	 Validation precision@k20: 0.77, accuracy@k20: 0.76
Epoch: 2 	 Validation precision@k25: 0.82, accuracy@k25: 0.82
Epoch: 2 	 Validation precision@k30: 0.86, accuracy@k30: 0.86
CPU: 15.20
RAM %: 59.9
Epoch: 3 	 Training Loss: 39.102801
Epoch: 3 	 Validation precision@k5: 0.73, accuracy@k5: 0.39
Epoch: 3 	 Validation p

CPU: 17.86
RAM %: 61.1
Epoch: 20 	 Training Loss: 34.555583
Epoch: 20 	 Validation precision@k5: 0.86, accuracy@k5: 0.47
Epoch: 20 	 Validation precision@k10: 0.83, accuracy@k10: 0.71
Epoch: 20 	 Validation precision@k15: 0.86, accuracy@k15: 0.83
Epoch: 20 	 Validation precision@k20: 0.90, accuracy@k20: 0.90
Epoch: 20 	 Validation precision@k25: 0.94, accuracy@k25: 0.94
Epoch: 20 	 Validation precision@k30: 0.96, accuracy@k30: 0.96
CPU: 30.66
RAM %: 61.2
Epoch: 21 	 Training Loss: 34.361229
Epoch: 21 	 Validation precision@k5: 0.86, accuracy@k5: 0.47
Epoch: 21 	 Validation precision@k10: 0.84, accuracy@k10: 0.72
Epoch: 21 	 Validation precision@k15: 0.87, accuracy@k15: 0.84
Epoch: 21 	 Validation precision@k20: 0.91, accuracy@k20: 0.90
Epoch: 21 	 Validation precision@k25: 0.94, accuracy@k25: 0.94
Epoch: 21 	 Validation precision@k30: 0.96, accuracy@k30: 0.96
CPU: 30.45
RAM %: 61.4
Epoch: 22 	 Training Loss: 34.182402
Epoch: 22 	 Validation precision@k5: 0.87, accuracy@k5: 0.47
Epoch: 

Epoch: 38 	 Validation precision@k30: 0.99, accuracy@k30: 0.99
CPU: 27.55
RAM %: 61.7
Epoch: 39 	 Training Loss: 31.568220
Epoch: 39 	 Validation precision@k5: 0.93, accuracy@k5: 0.52
Epoch: 39 	 Validation precision@k10: 0.92, accuracy@k10: 0.79
Epoch: 39 	 Validation precision@k15: 0.94, accuracy@k15: 0.91
Epoch: 39 	 Validation precision@k20: 0.96, accuracy@k20: 0.96
Epoch: 39 	 Validation precision@k25: 0.98, accuracy@k25: 0.98
Epoch: 39 	 Validation precision@k30: 0.99, accuracy@k30: 0.99
CPU: 27.33
RAM %: 62.0
Epoch: 40 	 Training Loss: 31.452974
Epoch: 40 	 Validation precision@k5: 0.93, accuracy@k5: 0.52
Epoch: 40 	 Validation precision@k10: 0.92, accuracy@k10: 0.79
Epoch: 40 	 Validation precision@k15: 0.94, accuracy@k15: 0.91
Epoch: 40 	 Validation precision@k20: 0.97, accuracy@k20: 0.96
Epoch: 40 	 Validation precision@k25: 0.98, accuracy@k25: 0.98
Epoch: 40 	 Validation precision@k30: 0.99, accuracy@k30: 0.99
CPU: 27.05
RAM %: 62.0
Epoch: 41 	 Training Loss: 31.351469
Epoch

Epoch: 57 	 Validation precision@k25: 0.99, accuracy@k25: 0.99
Epoch: 57 	 Validation precision@k30: 0.99, accuracy@k30: 0.99
CPU: 26.11
RAM %: 62.0
Epoch: 58 	 Training Loss: 30.098470
Epoch: 58 	 Validation precision@k5: 0.96, accuracy@k5: 0.54
Epoch: 58 	 Validation precision@k10: 0.96, accuracy@k10: 0.83
Epoch: 58 	 Validation precision@k15: 0.97, accuracy@k15: 0.94
Epoch: 58 	 Validation precision@k20: 0.98, accuracy@k20: 0.98
Epoch: 58 	 Validation precision@k25: 0.99, accuracy@k25: 0.99
Epoch: 58 	 Validation precision@k30: 0.99, accuracy@k30: 0.99
CPU: 25.99
RAM %: 62.1
Epoch: 59 	 Training Loss: 30.056477
Epoch: 59 	 Validation precision@k5: 0.96, accuracy@k5: 0.54
Epoch: 59 	 Validation precision@k10: 0.96, accuracy@k10: 0.83
Epoch: 59 	 Validation precision@k15: 0.97, accuracy@k15: 0.94
Epoch: 59 	 Validation precision@k20: 0.98, accuracy@k20: 0.98
Epoch: 59 	 Validation precision@k25: 0.99, accuracy@k25: 0.99
Epoch: 59 	 Validation precision@k30: 0.99, accuracy@k30: 0.99
CP

Epoch: 76 	 Validation precision@k20: 0.99, accuracy@k20: 0.99
Epoch: 76 	 Validation precision@k25: 0.99, accuracy@k25: 0.99
Epoch: 76 	 Validation precision@k30: 1.00, accuracy@k30: 1.00
CPU: 46.78
RAM %: 57.5
Epoch: 77 	 Training Loss: 29.403412
Epoch: 77 	 Validation precision@k5: 0.97, accuracy@k5: 0.55
Epoch: 77 	 Validation precision@k10: 0.97, accuracy@k10: 0.84
Epoch: 77 	 Validation precision@k15: 0.98, accuracy@k15: 0.95
Epoch: 77 	 Validation precision@k20: 0.99, accuracy@k20: 0.99
Epoch: 77 	 Validation precision@k25: 0.99, accuracy@k25: 0.99
Epoch: 77 	 Validation precision@k30: 1.00, accuracy@k30: 1.00
CPU: 46.75
RAM %: 57.6
Epoch: 78 	 Training Loss: 29.383380
Epoch: 78 	 Validation precision@k5: 0.98, accuracy@k5: 0.55
Epoch: 78 	 Validation precision@k10: 0.98, accuracy@k10: 0.84
Epoch: 78 	 Validation precision@k15: 0.98, accuracy@k15: 0.95
Epoch: 78 	 Validation precision@k20: 0.99, accuracy@k20: 0.99
Epoch: 78 	 Validation precision@k25: 0.99, accuracy@k25: 0.99
Ep

Epoch: 95 	 Validation precision@k15: 0.99, accuracy@k15: 0.96
Epoch: 95 	 Validation precision@k20: 0.99, accuracy@k20: 0.99
Epoch: 95 	 Validation precision@k25: 1.00, accuracy@k25: 1.00
Epoch: 95 	 Validation precision@k30: 1.00, accuracy@k30: 1.00
CPU: 46.23
RAM %: 60.3
Epoch: 96 	 Training Loss: 28.986934
Epoch: 96 	 Validation precision@k5: 0.98, accuracy@k5: 0.56
Epoch: 96 	 Validation precision@k10: 0.98, accuracy@k10: 0.85
Epoch: 96 	 Validation precision@k15: 0.99, accuracy@k15: 0.96
Epoch: 96 	 Validation precision@k20: 0.99, accuracy@k20: 0.99
Epoch: 96 	 Validation precision@k25: 1.00, accuracy@k25: 1.00
Epoch: 96 	 Validation precision@k30: 1.00, accuracy@k30: 1.00
CPU: 45.09
RAM %: 60.4
Epoch: 97 	 Training Loss: 28.976161
Epoch: 97 	 Validation precision@k5: 0.98, accuracy@k5: 0.56
Epoch: 97 	 Validation precision@k10: 0.98, accuracy@k10: 0.85
Epoch: 97 	 Validation precision@k15: 0.99, accuracy@k15: 0.96
Epoch: 97 	 Validation precision@k20: 0.99, accuracy@k20: 0.99
Ep

In [33]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(baseline_rnn, val_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.5799, accuracy@k5: 0.3228
Validation precision@k10: 0.5775, accuracy@k10: 0.4986
Validation precision@k15: 0.6306, accuracy@k15: 0.6099
Validation precision@k20: 0.6937, accuracy@k20: 0.6901
Validation precision@k25: 0.7456, accuracy@k25: 0.7452
Validation precision@k30: 0.7879, accuracy@k30: 0.7879
